In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../TextFiles/smsspamcollection.tsv', sep='\t')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 4 columns):
label      5572 non-null object
message    5572 non-null object
length     5572 non-null int64
punct      5572 non-null int64
dtypes: int64(2), object(2)
memory usage: 174.2+ KB


In [3]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [4]:
df.isnull()

,label,message,length,punct
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
5567,False,False,False,False
5568,False,False,False,False
5569,False,False,False,False
5570,False,False,False,False


In [5]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [6]:
len(df)

5572

In [7]:
df['label'].unique()

array(['ham', 'spam'], dtype=object)

In [8]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X = df['message']

In [11]:
y = df['label']

In [16]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
count_vect = CountVectorizer()

In [20]:
x_train_counts = count_vect.fit_transform(x_train)

In [21]:
x_train_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [22]:
x_train.shape

(3733,)

In [23]:
x_train_counts.shape

(3733, 7082)

In [24]:
from sklearn.feature_extraction.text import TfidfTransformer

In [25]:
tfidf_transformer = TfidfTransformer()

In [28]:
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

In [29]:
x_train_tfidf.shape

(3733, 7082)

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
vectorizer = TfidfVectorizer()

In [32]:
x_train_tfidf = vectorizer.fit_transform(x_train)

In [33]:
from sklearn.svm import LinearSVC

In [35]:
clf = LinearSVC()

In [36]:
clf.fit(x_train_tfidf, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [37]:
from sklearn.pipeline import Pipeline

In [38]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [39]:
text_clf.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [41]:
predictions = text_clf.predict(x_test)

In [42]:
from sklearn.metrics import confusion_matrix, classification_report

In [43]:
print(confusion_matrix(y_test, predictions))

[[1586    7]
 [  12  234]]


In [44]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [45]:
from sklearn import metrics

In [47]:
metrics.accuracy_score(y_test, predictions)

0.989668297988037

In [51]:
text_clf.predict(["Free laptop at Sillicon Valley"])

array(['ham'], dtype=object)

In [52]:
text_clf.predict(["Congratulations! You've been selected as a winner. Text WON to 09424352 congratulations free entry to contest!"])

array(['spam'], dtype=object)